In [23]:
import pandas as pd
import numpy as np
import nltk

In [6]:
data = pd.read_csv('products.csv', error_bad_lines=False, warn_bad_lines=True)


<ipython-input-6-c2ec9e6fb549>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('products.csv', error_bad_lines=False, warn_bad_lines=True)
<ipython-input-6-c2ec9e6fb549>:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('products.csv', error_bad_lines=False, warn_bad_lines=True)


In [7]:
data.head(2)

,name,category,price,image,Unnamed: 4,description,Unnamed: 6,Unnamed: 7,brand
0,Alisha Solid Women's Cycling Shorts,"[""clothing,""women clothing""]",999.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",NaN,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",32157.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",NaN,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,NaN,NaN,FabHomeDecor


In [9]:
columns_to_drop = ['Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7']
data_cleaned = data.drop(columns=columns_to_drop)
data_cleaned.head(2)

,name,category,price,image,description,brand
0,Alisha Solid Women's Cycling Shorts,"[""clothing,""women clothing""]",999.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",32157.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor


In [10]:
data_cleaned.isnull().sum()

name              0
category          0
price            78
image             3
description       2
brand          5864
dtype: int64

In [11]:
data_cleaned = data_cleaned.dropna()
data_cleaned.head(2)

,name,category,price,image,description,brand
0,Alisha Solid Women's Cycling Shorts,"[""clothing,""women clothing""]",999.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",32157.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor


In [12]:
data_cleaned.isnull().sum()

name           0
category       0
price          0
image          0
description    0
brand          0
dtype: int64

In [17]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stemmed = [stemmer.stem(w) for w in tokens ]
    return " ".join(stemmed)
    
    

In [33]:
import re



In [34]:
# data_cleaned['stemmed_tokens'] = data_cleaned.apply(lambda row:tokenize_stem(row['name'] + " " + row['description']), axis=1)



import re

def tokenize_stem(text):

    pattern = r'\W+'  
    tokens = re.split(pattern, text.lower()) 
   
    return " ".join(tokens)

data_cleaned['stemmed_tokens'] = data_cleaned.apply(lambda row: tokenize_stem(row['name'] + " " + row['description']), axis=1)


In [36]:
data_cleaned.head()


,name,category,price,image,description,brand,stemmed_tokens
0,Alisha Solid Women's Cycling Shorts,"[""clothing,""women clothing""]",999.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,alisha solid women s cycling shorts key featur...
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",32157.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,fabhomedecor fabric double sofa bed fabhomedec...
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",999.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,aw bellies key features of aw bellies sandals ...
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",699.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,alisha solid women s cycling shorts key featur...
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",220.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,sicons all purpose arnica dog shampoo specific...


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
tfidfv = TfidfVectorizer(tokenizer = tokenize_stem )
def cosine_sim(txt1, txt2):
    matrix = tfidfv.fit_transform([txt1, txt2])
    return cosine_similarity(matrix)[0, 1]  

In [55]:
def search_product(query):
    stemmed_query = tokenize_stem(query)
    data_cleaned['similarity'] = data_cleaned['stemmed_tokens'].apply(lambda x: cosine_sim(stemmed_query, x))
    res = data_cleaned.sort_values(by='similarity', ascending=False).head(10)[['name', 'description', 'price', 'category', 'image', 'brand']]
    return res

In [59]:
search_product('AW Bellies')

/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,name,description,price,category,image,brand
4346,Affinity Jewellers Designer Silver Bead White ...,Affinity Jewellers Designer Silver Bead White ...,1624.0,"[""Jewellery >> Necklaces & Chains >> Necklaces""]","[""http://img5a.flixcart.com/image/necklace-cha...",Affinity Jewellers
9509,Shashvat Jewels Designer & Elegant Silver Jewe...,Shashvat Jewels Designer & Elegant Silver Jewe...,6248.0,"[""Jewellery >> Jewellery Sets""]","[""http://img6a.flixcart.com/image/jewellery-se...",Shashvat Jewels
10417,LATHERO Full Sleeve Self Design Women's Jacket,Specifications of LATHERO Full Sleeve Self Des...,12500.0,"[""Clothing >> Women's Clothing >> Western Wear...","[""http://img6a.flixcart.com/image/jacket/j/h/t...",LATHERO
4654,Ada Jewel Alloy Necklace,Ada Jewel Alloy Necklace\n ...,1298.0,"[""Jewellery >> Necklaces & Chains >> Necklaces""]","[""http://img6a.flixcart.com/image/necklace-cha...",Ada Jewel
15556,Allen Solly Self Design Crew Neck Boy's Sweater,Specifications of Allen Solly Self Design Crew...,1599.0,"[""Clothing >> Kids' Clothing >> Boys Wear >> W...","[""http://img6a.flixcart.com/image/sweater/q/m/...",Allen Solly
11097,Classique Designer Jewellery Beautiful Alloy J...,Classique Designer Jewellery Beautiful Alloy J...,1999.0,"[""Jewellery >> Jewellery Sets""]","[""http://img5a.flixcart.com/image/jewellery-se...",Classique Designer Jewellery
10,Ladela Bellies,Key Features of Ladela Bellies Brand: LADELA C...,1724.0,"[""Footwear >> Women's Footwear >> Ballerinas >...","[""http://img5a.flixcart.com/image/shoe/s/g/m/b...",Ladela
13656,"Sheetal Jewellery Brass, Alloy Jewel Set","Sheetal Jewellery Brass, Alloy Jewel Set - Buy...",800.0,"[""Jewellery >> Jewellery Sets""]","[""http://img5a.flixcart.com/image/jewellery-se...",Sheetal Jewellery
18199,"Sheetal Jewellery Brass, Alloy Jewel Set","Sheetal Jewellery Brass, Alloy Jewel Set - Buy...",1200.0,"[""Jewellery >> Jewellery Sets""]","[""http://img6a.flixcart.com/image/jewellery-se...",Sheetal Jewellery
14596,SKOOKIE Sleeveless Solid Men's Jacket,Key Features of SKOOKIE Sleeveless Solid Men's...,1599.0,"[""Clothing >> Men's Clothing >> Winter & Seaso...","[""http://img6a.flixcart.com/image/jacket/2/j/a...",SKOOKIE


In [62]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import warnings

# Suppress the specific UserWarning from sklearn
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# Define your tokenize_stem function using regular expressions
def tokenize_stem(text):
    if pd.isnull(text):
        text = ""
    pattern = r'\W+'  # Match one or more non-word characters
    tokens = re.split(pattern, text.lower())  # Split text into tokens and convert to lowercase
    return " ".join(tokens)

# Initialize TfidfVectorizer with the custom tokenizer
tfidfv = TfidfVectorizer(tokenizer=tokenize_stem)

# Define the cosine similarity function
def cosine_sim(txt1, txt2):
    matrix = tfidfv.fit_transform([txt1, txt2])
    return cosine_similarity(matrix)[0, 1]

# Define the search_product function
def search_product(query):
    stemmed_query = tokenize_stem(query)
    data_cleaned['similarity'] = data_cleaned['stemmed_tokens'].apply(lambda x: cosine_sim(stemmed_query, x))
    res = data_cleaned.sort_values(by='similarity', ascending=False).head(10)[['name', 'description', 'price', 'category', 'image', 'brand']]
    return res

# Example call to the function
print(search_product('Ladela Bellies'))


                                                    name  \
4346   Affinity Jewellers Designer Silver Bead White ...   
10417     LATHERO Full Sleeve Self Design Women's Jacket   
17695            INDIRANG Sleeveless Solid Girl's Jacket   
7222                                  GIA Metal Necklace   
10                                        Ladela Bellies   
4463   Affinity Jewellers Designer Gold Hyderabadi Go...   
5387   Allen Solly Solid Single Breasted Casual Boy's...   
4338   Affinity Jewellers Designer Meena Bead Red Taa...   
9509   Shashvat Jewels Designer & Elegant Silver Jewe...   
5782   Hotpiper White Floral Design Pearl Yellow Gold...   

                                             description    price  \
4346   Affinity Jewellers Designer Silver Bead White ...   1624.0   
10417  Specifications of LATHERO Full Sleeve Self Des...  12500.0   
17695  Specifications of INDIRANG Sleeveless Solid Gi...    799.0   
7222   GIA Metal Necklace\n                         P...    499